In [1]:
from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
import pandas as pd
from collections import OrderedDict
from datetime import date
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import fiona
import shapely
import datetime

In [2]:
"""
AOI Parameters
"""

# bbox
bbox = (-4.860230,47.428380,-0.982056,48.897512)
bbox_poly = shapely.geometry.box(*bbox, ccw=True)


# start-end times
start = datetime.datetime(2018,4,1)
end = datetime.datetime(2018,9,30)

In [3]:
# define tiles
tiles = ['30UUU', '30UVV', '30UVU', '30TVT', '30UWU', '30TWT', '30UXU']


# set up API
user = "sdonike" ####### ## change this!
pw =  "Iha0504G$" ######## ## change this!
api = SentinelAPI(user, pw, api_url='https://scihub.copernicus.eu/dhus', show_progressbars= True,timeout=100)


In [14]:
def get_footprints(tiles,lower_cloud,upper_cloud):
    # set query params
    query_kwargs = {
        'platformname': 'Sentinel-2',
        #'producttype': 'S2MSI2Ap',
        #'processinglevel': 'Level-1C',
        'date': ('20210401', '20210930'),
        'cloudcoverpercentage': (lower_cloud, upper_cloud),
         'limit': 999} 
    
    #products_ls = []
    footprints = []

    products = OrderedDict()
    for tile in tiles:
        products = OrderedDict()
        kw = query_kwargs.copy()
        kw['tileid'] = tile  
        pp = api.query(**kw)
        products.update(pp)
        #products_ls.append(products)
        #print("Tile:",tile,"\t - Products:",len(products))


        products_df = api.to_dataframe(products)
        products_df_sorted = products_df.sort_values(['size'],ascending = [False])
        #products_df_sorted = products_df_sorted.head (1)

        #print(products_df_sorted['size'])
        #print(products_df_sorted['title'])

        fp = []
        for i in products_df_sorted["footprint"]:
            fp.append(i)

        #tmp_ls = [products_df_sorted["title"],products_df_sorted["tileid"],products_df_sorted["footprint"],products_df_sorted["datatakesensingstart"]]

        footprints.append(fp)


        #download data
        # api.download_all(products_df_sorted.index, directory_path= r'C:\Users\EQuilibrium\PyBegining\downloader\Downld_Sen',max_attempts=2, checksum = True )
        #api.download(products_df_sorted.index, directory_path= r'C:\Users\EQuilibrium\PyBegining\downloader\Downld_Sen', checksum = True )
        #print(products_df_sorted)
    # flatten list
    footprints = [item for sublist in footprints for item in sublist]
    footprints_df = pd.DataFrame(footprints,columns=["geometry"])
    #footprints_df
    
    footprints_df['geometry'] = gpd.GeoSeries.from_wkt(footprints_df['geometry'])
    gdf = gpd.GeoDataFrame(footprints_df, geometry='geometry')
    print(footprin)
    return(gdf)

In [15]:

get_footprints(tiles,0, 5).to_file('qgis_files/sen2_tiles/footprints_05.shp')
get_footprints(tiles,0,10).to_file('qgis_files/sen2_tiles/footprints_10.shp')  
get_footprints(tiles,0,15).to_file('qgis_files/sen2_tiles/footprints_15.shp')  
get_footprints(tiles,0,20).to_file('qgis_files/sen2_tiles/footprints_20.shp')  

Tile: 30UUU 	 - Products: 34
Tile: 30UVV 	 - Products: 27
Tile: 30UVU 	 - Products: 15
Tile: 30TVT 	 - Products: 27
Tile: 30UWU 	 - Products: 25
Tile: 30TWT 	 - Products: 35
Tile: 30UXU 	 - Products: 14


,geometry
0,"MULTIPOLYGON (((-4.52032 47.75351, -4.20361 47..."
1,"MULTIPOLYGON (((-4.51111 47.75367, -4.20361 47..."
2,"MULTIPOLYGON (((-4.51593 47.75359, -4.20361 47..."
3,"MULTIPOLYGON (((-4.51260 47.75364, -4.20361 47..."
4,"MULTIPOLYGON (((-4.50519 47.75377, -4.20361 47..."
...,...
172,"MULTIPOLYGON (((-0.43805 47.73530, -0.39688 47..."
173,"MULTIPOLYGON (((-0.44330 47.73539, -0.43701 47..."
174,"MULTIPOLYGON (((-0.43945 47.73532, -0.38443 47..."
175,"MULTIPOLYGON (((-0.43118 47.73517, -0.41431 47..."
